# Heatmap for Demand
In this notebook we plot a Choropleth for estimated cash demand through a mixed effects linear model, that takes into account Eurostat economic and demographic data, as well as concentration of interest points.
<br> The analysis is done at NUTS2 level, because of the important amount of missing values in SUCH and SPACE data at lower granularities.

In [1]:
import s3fs
fs = s3fs.S3FileSystem(anon=False)

In [2]:
import boto3
s3 = boto3.client("s3")

In [3]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize']=(5,5)
plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_colwidth', -1)

# Set of homemade functions for automatic a bit the plotting
def our_ax():
    ax = plt.axes()
    ax.grid(False)
    ax.set_facecolor('white')
    return ax

def tempp(df, var):
    temp = df[[var]]
    temp["geometry"] = temp[var]
    temp = gpd.GeoDataFrame(temp)
    return temp

def plot_geom(geom, color): 
    ax = plt.axes()
    ax.grid(False)
    ax.set_facecolor('white')
    if geom.geom_type == 'Polygon':
        return plt.plot(*geom.exterior.xy, color=color)
    elif geom.geom_type == 'Point':
        return plt.scatter(geom.x, geom.y, marker='o', color=color)
                       
def plot_two(one, two): 
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    one.plot(ax=our_ax(), color='white', edgecolor='orange')
    two.plot(ax=our_ax(), color="white",edgecolor='darkblue')
    return plt.show();

In [4]:
import geopandas as gpd

In [6]:
projection="4326"
level="_LEVL_2"
eurostatfile="NUTS2016-20M/NUTS_RG_20M_2016_"+projection+level
nutssourcefile="s3://ecb-hackathon-202006-dbn-shared/data/eurostat/nuts/"+eurostatfile+".geojson"

In [7]:
neighborhoods = gpd.read_file(nutssourcefile)

In [8]:
neighborhoods['VALUE'] = np.random.rand(neighborhoods.shape[0])

In [16]:
predictions = pd.read_csv("s3://ecb-hackathon-202006-dbn-group1/data/test/CashDemandPrediction_TotValue_LinearModel.csv")

In [17]:
predictions.drop('Unnamed: 0', axis=1, inplace=True)

In [21]:
predictions.rename(columns={'NUTS2':'id', 'Tot_v':'VALUE'}, inplace=True)

In [24]:
### AREAS
# Prepare the data from the spatial join performed earlier.
import folium
import json


grp = predictions[['id','VALUE']].copy()

fmap = folium.Map(tiles="Stamen Terrain", 
                  location=([48.86750, 2.36383]), 
                  zoom_start=4)

with fs.open(nutssourcefile) as f:
      data = json.load(f)
        
folium.Choropleth(
  geo_data=data,
  name="EU Cash Demand",
  data=grp,
  columns=['id', 'VALUE'],
  key_on='id',
  fill_color="YlOrRd",
  fill_opacity=0.7,
  line_opacity=0.5,
  legend_name=("Colour gradient depending on the estimated cash demand."),
  reset =True
).add_to(fmap)


folium.LayerControl().add_to(fmap)

fmap

In [31]:
with fs.open(nutssourcefile) as f:
      data = json.load(f)

In [23]:
predictions.VALUE = np.log(predictions.VALUE)